In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo

## Extract Victorian Biodiversity Atlas (VBA) fauna data

In [2]:
# Read just the column names in fauna data csv
col_names = pd.read_csv("../data/VBA_2015_2020.csv", nrows = 0).columns
col_names

Index(['RECORD_ID', 'SITE_ID', 'SURVEY_ID', 'PROJECT_ID', 'TAXON_ID',
       'SCI_NAME', 'COMM_NAME', 'RECORDTYPE', 'RELIABILTY', 'TOTALCOUNT',
       'STARTDATE', 'START_YEAR', 'START_MTH', 'ENDDATE', 'END_YEAR',
       'END_MTH', 'LOCN_DESC', 'TAXON_TYPE', 'LONG_DD94', 'LAT_DD94'],
      dtype='object')

In [3]:
# Set data types for columns with data types other than strings
dtypes_dict = {
    "TOTALCOUNT": int,
    "START_YEAR": int,
    "START_MTH": int,
    "END_YEAR": int,
    "END_MTH": int,
    "LONG_DD94": float,
    "LAT_DD94": float
}

In [4]:
# Read in vic fauna csv from 2015 to 2020
fauna_data = pd.read_csv(
    "../data/VBA_2015_2020.csv",
    parse_dates = ["STARTDATE", "ENDDATE"],
    dtype = {col: str for col in col_names if col not in dtypes_dict})
fauna_data.head()

,RECORD_ID,SITE_ID,SURVEY_ID,PROJECT_ID,TAXON_ID,SCI_NAME,COMM_NAME,RECORDTYPE,RELIABILTY,TOTALCOUNT,STARTDATE,START_YEAR,START_MTH,ENDDATE,END_YEAR,END_MTH,LOCN_DESC,TAXON_TYPE,LONG_DD94,LAT_DD94
0,8597419,947931,1405903,4377,1557,Paratya australiensis,Common Freshwater Shrimp,Observation,Confirmed,0,2016-12-16,2016,12,NaT,0,0,McCallum Creek-4_7-TR-16-333,"Mussels, decopod crustacea",143.649002,-37.283901
1,9067844,1084677,1776514,5326,10408,Colluricincla harmonica,Grey Shrike-thrush,Observation with supporting evidence,Acceptable,0,2018-08-28,2018,8,2018-09-25,2018,9,345-513-0003 FSQ1,Passerine birds,145.768997,-37.782501
2,8218590,771970,1221401,4366,10991,Turdus merula,Common Blackbird,Seen,Acceptable,0,2015-04-12,2015,4,NaT,0,0,Ocean Acres Bush Park Nature Reserve,Passerine birds,144.287399,-38.315601
3,9047388,1070861,1760792,5326,11242,Wallabia bicolor,Black-tailed Wallaby,Observation with supporting evidence,Acceptable,0,2018-11-14,2018,11,2018-12-14,2018,12,833-518-0004 BUQ1,Mammals,148.848099,-37.601601
4,9359539,1116727,1809370,5543,10525,Cisticola exilis,Golden-headed Cisticola,Seen,Acceptable,1,2015-05-07,2015,5,2015-05-07,2015,5,MANNIBADAR (581481),Passerine birds,143.481903,-37.781799


## Transform VBA fauna data

In [5]:
# Column Renaming
fauna_df = fauna_data.rename(columns={
    "RECORD_ID": "record_id",
    "SITE_ID": "site_id",
    "SURVEY_ID": "survey_id",
    "PROJECT_ID": "project_id",
    "TAXON_ID": "taxon_id",
    "SCI_NAME": "sci_name",
    "COMM_NAME": "comm_name",
    "RECORDTYPE": "recordtype",
    "RELIABILTY": "reliability",
    "TOTALCOUNT": "totalcount",
    "STARTDATE": "start_date",
    "START_YEAR": "start_year",
    "START_MTH": "start_mth",
    "ENDDATE": "end_date",
    "END_YEAR": "end_year",
    "END_MTH": "end_mth",
    "LOCN_DESC": "location_desc",
    "TAXON_TYPE": "taxon_type",
    "LONG_DD94": "long",
    "LAT_DD94": "lat"})

In [6]:
# Test record_id uniqueness
fauna_df.record_id.is_unique

True

In [7]:
print(f"Number of unique record ids: {fauna_df.record_id.nunique()}")

Number of unique record ids: 346829


In [8]:
# Test survey_id uniqueness
fauna_df.survey_id.is_unique

False

In [9]:
print(f"Number of unique survey ids: {fauna_df.survey_id.nunique()}")

Number of unique survey ids: 97240


In [10]:
# Test site_id uniqueness
fauna_df.site_id.is_unique

False

In [11]:
print(f"Number of unique site ids: {fauna_df.site_id.nunique()}")

Number of unique site ids: 55164


In [12]:
# Test project_id uniqueness
fauna_df.project_id.is_unique

False

In [13]:
print(f"Number of unique project ids: {fauna_df.project_id.nunique()}")

Number of unique project ids: 522


In [14]:
# Test taxon_id uniqueness
fauna_df.taxon_id.is_unique

False

In [15]:
print(f"Number of unique taxon ids: {fauna_df.taxon_id.nunique()}")

Number of unique taxon ids: 941


In [16]:
print(f"Number of unique taxon types: {fauna_df.taxon_type.nunique()}")

Number of unique taxon types: 14


In [17]:
# Reorder the columns
fauna_df = fauna_df[["record_id", "survey_id", "site_id", "project_id", "taxon_id", "taxon_type"
                     ,"comm_name", "sci_name", "totalcount", "location_desc", "long", "lat"
                     ,"end_year", "end_mth", "end_date", "start_year", "start_mth", "start_date"
                     ,"recordtype", "reliability"]]
fauna_df.head()

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
0,8597419,1405903,947931,4377,1557,"Mussels, decopod crustacea",Common Freshwater Shrimp,Paratya australiensis,0,McCallum Creek-4_7-TR-16-333,143.649002,-37.283901,0,0,NaT,2016,12,2016-12-16,Observation,Confirmed
1,9067844,1776514,1084677,5326,10408,Passerine birds,Grey Shrike-thrush,Colluricincla harmonica,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
2,8218590,1221401,771970,4366,10991,Passerine birds,Common Blackbird,Turdus merula,0,Ocean Acres Bush Park Nature Reserve,144.287399,-38.315601,0,0,NaT,2015,4,2015-04-12,Seen,Acceptable
3,9047388,1760792,1070861,5326,11242,Mammals,Black-tailed Wallaby,Wallabia bicolor,0,833-518-0004 BUQ1,148.848099,-37.601601,2018,12,2018-12-14,2018,11,2018-11-14,Observation with supporting evidence,Acceptable
4,9359539,1809370,1116727,5543,10525,Passerine birds,Golden-headed Cisticola,Cisticola exilis,1,MANNIBADAR (581481),143.481903,-37.781799,2015,5,2015-05-07,2015,5,2015-05-07,Seen,Acceptable


In [18]:
# Overview of the fauna data
fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346829 entries, 0 to 346828
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   record_id      346829 non-null  object        
 1   survey_id      346829 non-null  object        
 2   site_id        346829 non-null  object        
 3   project_id     346829 non-null  object        
 4   taxon_id       346829 non-null  object        
 5   taxon_type     346829 non-null  object        
 6   comm_name      346829 non-null  object        
 7   sci_name       346829 non-null  object        
 8   totalcount     346829 non-null  int64         
 9   location_desc  346829 non-null  object        
 10  long           346829 non-null  float64       
 11  lat            346829 non-null  float64       
 12  end_year       346829 non-null  int64         
 13  end_mth        346829 non-null  int64         
 14  end_date       52465 non-null   datetime64[ns]
 15  

In [19]:
# We can see that the TOTALCOUNT of some records is 0. Let's have an overview of them
zero_totalcount = fauna_df[fauna_df["totalcount"] == 0]
zero_totalcount.head(10)

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
0,8597419,1405903,947931,4377,1557,"Mussels, decopod crustacea",Common Freshwater Shrimp,Paratya australiensis,0,McCallum Creek-4_7-TR-16-333,143.649002,-37.283901,0,0,NaT,2016,12,2016-12-16,Observation,Confirmed
1,9067844,1776514,1084677,5326,10408,Passerine birds,Grey Shrike-thrush,Colluricincla harmonica,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
2,8218590,1221401,771970,4366,10991,Passerine birds,Common Blackbird,Turdus merula,0,Ocean Acres Bush Park Nature Reserve,144.287399,-38.315601,0,0,NaT,2015,4,2015-04-12,Seen,Acceptable
3,9047388,1760792,1070861,5326,11242,Mammals,Black-tailed Wallaby,Wallabia bicolor,0,833-518-0004 BUQ1,148.848099,-37.601601,2018,12,2018-12-14,2018,11,2018-11-14,Observation with supporting evidence,Acceptable
6,9067841,1776514,1084677,5326,10488,Passerine birds,White-browed Scrubwren,Sericornis frontalis,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
9,8897604,1686135,1000965,4335,528552,Mammals,Red Fox,Vulpes vulpes,0,Basalt 10,144.098206,-37.270500,2018,2,2018-02-07,2018,1,2018-01-17,Observation with supporting evidence,Acceptable
10,9345216,1799930,1107289,2936,5140,Fish,Dry waterbody,Misc Dry,0,Pig and Whistle Creek-2_23-TR-18-421,147.853394,-37.532299,0,0,NaT,2018,11,2018-11-16,Observation,Confirmed
12,8432377,1292811,840246,4551,10364,Passerine birds,Willie Wagtail,Rhipidura leucophrys,0,Bulla Hill and School Hill,144.802200,-37.634102,0,0,NaT,2015,5,2015-05-18,Observation,Acceptable
13,8956790,1715391,1027944,4836,11003,Mammals,Short-beaked Echidna,Tachyglossus aculeatus,0,New Holland Mouse camera survey_Site_NHM423,147.533798,-38.084000,2018,3,2018-03-20,2018,3,2018-03-07,Observation with supporting evidence,Acceptable
15,8994856,1741637,1052493,5326,11115,Mammals,Mountain Brush-tailed Possum,Trichosurus cunninghami,0,298-516-0003,145.521393,-37.442902,2018,8,2018-08-23,2018,7,2018-07-27,Observation with supporting evidence,Acceptable


In [20]:
# Percentage of number of rows with totalcount equal to 0 against total number of rows of the dataframe
(zero_totalcount.shape[0]/fauna_df.shape[0])*100

19.936914156544002

## Filter VBA fauna data against scraped data

In [21]:
# Import the webscraped animal data
scraped_df = pd.read_csv("../data/animal_image_to_merge.csv", dtype="str")
scraped_df.head()

,animal_name,image_url,image_alternative,introduction,threat_paragraph
0,Alpine She-oak Skink,https://ZoosVic-Endpoint-Blob-Prod.azureedge.n...,Alpine She-oak Skink sunning it self on a rock...,Found in only a few locations in Victoria and ...,Major threats\nFire is a huge danger to the Al...
1,Baw Baw Frog,https://www.zoo.org.au/media/2052/21295_baw_ba...,Baw Baw Frog resting in bright green moss. Loo...,All estimates point to extinction in the wild ...,Major threats\nThe loss of the Baw Baw Frog is...
2,Brush-tailed Rock-wallaby,https://www.zoo.org.au/media/2045/21882_brush-...,Brush Tailed Rock Wallabies resting in the grass.,"In Victoria, the Brush-tailed Rock-wallaby now...",Major threats \nChanges to habitat and the imp...
3,Eastern Barred Bandicoot,https://www.zoo.org.au/media/2053/4376_eastern...,Small Eastern Barred Bandicoot side view forag...,The Eastern Barred Bandicoot is listed as exti...,The plan for recovery\nZoos Victoria has partn...
4,Giant Burrowing Frog,https://www.zoo.org.au/media/2056/23479_giant_...,Giant Burrowing Frog on wet rocks side view. T...,Although we know that populations of the Giant...,"['Over the next five years, Zoos Victoria will..."


In [22]:
# Extract list of unique animals of interest
species = scraped_df["animal_name"].unique().tolist()

In [23]:
# Filter the fauna data with the species of interest
short_fauna_df = fauna_df[fauna_df["comm_name"].isin(species)]
short_fauna_df.head()

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
102,8261909,1225877,833395,4236,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,0,DSS3B,145.841904,-37.845600,0,0,NaT,2016,4,2016-04-24,Observation,Confirmed
195,8590335,1401349,943395,4095,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,DELWP Case Reference number 2017-0060,146.288498,-37.922600,0,0,NaT,2017,8,2017-08-20,Seen,Confirmed
391,6942095,1102090,766516,4078,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,Mon1_camera1,145.934403,-37.818600,2015,9,2015-09-29,2015,9,2015-09-08,Observation,Confirmed
426,8596613,1405576,947609,1,10309,Non-passerine birds,Swift Parrot,Lathamus discolor,5,"Mclaughlans Lane Pipetrack, Plenty",145.108398,-37.676399,2017,9,2017-09-25,2017,9,2017-09-25,Observation,Acceptable
538,6870927,1085373,716545,4078,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,40.2_camera2,146.115204,-37.784802,2015,5,2015-05-11,2015,4,2015-04-17,Observation,Confirmed


In [24]:
# Overview of the fauna data after filtering
short_fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2712 entries, 102 to 346720
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   record_id      2712 non-null   object        
 1   survey_id      2712 non-null   object        
 2   site_id        2712 non-null   object        
 3   project_id     2712 non-null   object        
 4   taxon_id       2712 non-null   object        
 5   taxon_type     2712 non-null   object        
 6   comm_name      2712 non-null   object        
 7   sci_name       2712 non-null   object        
 8   totalcount     2712 non-null   int64         
 9   location_desc  2712 non-null   object        
 10  long           2712 non-null   float64       
 11  lat            2712 non-null   float64       
 12  end_year       2712 non-null   int64         
 13  end_mth        2712 non-null   int64         
 14  end_date       899 non-null    datetime64[ns]
 15  start_year     27

In [25]:
# Check for any extreme values
print(f"Maximum total count is: {short_fauna_df.totalcount.max()}")
print(f"Minimum total count is: {short_fauna_df.totalcount.min()}")
print(f"Maximum longitude is: {short_fauna_df.long.max()}")
print(f"Minimum longitude is: {short_fauna_df.long.min()}")
print(f"Maximum latitude is: {short_fauna_df.lat.max()}")
print(f"Minimum latitude is: {short_fauna_df.lat.min()}")
print(f"Maximum end year is: {short_fauna_df.end_year.max()}")
print(f"Minimum end year is: {short_fauna_df.end_year.min()}")
print(f"Maximum end month is: {short_fauna_df.end_mth.max()}")
print(f"Minimum end month is: {short_fauna_df.end_mth.min()}")
print(f"Maximum start year is: {short_fauna_df.start_year.max()}")
print(f"Minimum start year is: {short_fauna_df.start_year.min()}")
print(f"Maximum start month is: {short_fauna_df.start_mth.max()}")
print(f"Minimum start month is: {short_fauna_df.start_mth.min()}")

Maximum total count is: 128
Minimum total count is: 0
Maximum longitude is: 149.9367981
Minimum longitude is: 140.9933014
Maximum latitude is: -34.5940018
Minimum latitude is: -39.0321007
Maximum end year is: 2020
Minimum end year is: 0
Maximum end month is: 12
Minimum end month is: 0
Maximum start year is: 2020
Minimum start year is: 2015
Maximum start month is: 12
Minimum start month is: 1


As can be seen, there are a number of records with total count of 0. They are records of surveys with no sightings of a targeted specie. Hence, we'll remove them.

As there are a lot of null end_date values, their extracted end years and end months equal to 0. Hence we might use start date in our time series visualisation. The null end dates might indicate that a survey hasn't ended up to our group's data extraction.

In [26]:
print(f"The number of records with totalcount of zero: {short_fauna_df[short_fauna_df.totalcount == 0].shape[0]}")

The number of records with totalcount of zero: 623


In [27]:
# Values in location description column
short_fauna_df["location_desc"].unique()

array(['DSS3B', 'DELWP Case Reference number 2017-0060', 'Mon1_camera1',
       ..., '316A', '316B', '316C'], dtype=object)

In [28]:
# Values in record types column
short_fauna_df["recordtype"].unique()

array(['Observation', 'Seen', 'Observation with supporting evidence',
       'Captured and released', 'Indirect evidence', 'Heard', 'Captured'],
      dtype=object)

In [29]:
# Values in reliability column
short_fauna_df["reliability"].unique()

array(['Confirmed', 'Acceptable'], dtype=object)

Values in the location description, record types and reliability columns do not seem to be informative enough. Hence we'll remove these columns. We'll also remove columns project_id and site_id as they are not required for our project's purpose.

In [30]:
# Filter out the records with total count of 0 and remove end_year, end_date, and end_mth columns
final_fauna_df = short_fauna_df[short_fauna_df.totalcount > 0].drop([
    'site_id', 'project_id', "location_desc", 'end_year', 'end_mth', 'end_date', 'recordtype', 'reliability'], axis = 1)
final_fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2089 entries, 195 to 346720
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   record_id   2089 non-null   object        
 1   survey_id   2089 non-null   object        
 2   taxon_id    2089 non-null   object        
 3   taxon_type  2089 non-null   object        
 4   comm_name   2089 non-null   object        
 5   sci_name    2089 non-null   object        
 6   totalcount  2089 non-null   int64         
 7   long        2089 non-null   float64       
 8   lat         2089 non-null   float64       
 9   start_year  2089 non-null   int64         
 10  start_mth   2089 non-null   int64         
 11  start_date  2089 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 212.2+ KB


## Filter webscraped animal image data against VBA fauna data 

In [31]:
final_animal_list = final_fauna_df["comm_name"].unique().tolist()
final_animal_list

["Leadbeater's Possum",
 'Swift Parrot',
 'Mountain Pygmy-possum',
 'New Holland Mouse',
 'Mallee Emu-wren',
 'Hooded Plover',
 'Giant Burrowing Frog',
 'Greater Glider',
 'Regent Honeyeater']

In [32]:
# Filter the webscraped data to have only the above animals
final_scraped_df = scraped_df[scraped_df["animal_name"].isin(final_animal_list)]
final_scraped_df

,animal_name,image_url,image_alternative,introduction,threat_paragraph
4,Giant Burrowing Frog,https://www.zoo.org.au/media/2056/23479_giant_...,Giant Burrowing Frog on wet rocks side view. T...,Although we know that populations of the Giant...,"['Over the next five years, Zoos Victoria will..."
11,Leadbeater's Possum,https://www.zoo.org.au/media/2057/22861_leadbe...,Close up view of the face of a Leadbeater Poss...,"Once thought to be extinct, the Leadbeater's P...",The major threats\nThe loss of hollow-bearing ...
13,Mallee Emu-wren,https://www.zoo.org.au/media/1961/23483_mallee...,Mallee Emu-wren in long dry grass looking at t...,The Mallee Emu-wren is particularly vulnerable...,"['In fact, it was a series of fires that cause..."
14,Mountain Pygmy-possum,https://www.zoo.org.au/media/2058/16910_mounta...,Mountain Pygmy Possum standing on its hind leg...,Mountain Pygmy-possums were thought to be exti...,"The major threats \nClimate change, the loss o..."
15,New Holland Mouse,https://www.zoo.org.au/media/1732/new-holland-...,New Holland Mouse getting a health check wrapp...,The New Holland Mouse is classified as extinct...,The major threats\nThe New Holland Mouse is in...
19,Regent Honeyeater,https://www.zoo.org.au/media/2055/22249_regent...,Regent Honeyeater bird on a branch looking dow...,The Regent Honeyeater has been in decline sinc...,The major threats\nThe loss of the Box-Ironbar...
25,Swift Parrot,https://www.zoo.org.au/media/1960/23484_swift_...,Green Swift Parrot perched on a branch looking...,The Swift Parrot is critically endangered.,['Unless we can solve the issues surrounding i...
27,Greater Glider,https://www.environment.vic.gov.au/__data/asse...,Greater Glider,NaN,Species InformationThe Greater Glider is Austr...
28,Hooded Plover,https://www.environment.vic.gov.au/__data/asse...,Hooded Plover,Species Information\n\nHooded Plovers are a re...,Threats\n\nHooded Plovers often share their oc...


## Load

In [33]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collections
db = client.animal_visual_db
vba_fauna = db.vba_fauna
scraped_fauna = db.scraped_fauna

In [34]:
# Drops collections if available to remove duplicates
vba_fauna.drop()
scraped_fauna.drop()

In [35]:
# Load vba fauna data into the vba_fauna collection
vba_fauna.insert_many(final_fauna_df.to_dict('records'))

In [36]:
# Load scraped fauna image and info into the scraped_fauna collection
scraped_fauna.insert_many(final_scraped_df.to_dict('records'))

In [37]:
from pprint import pprint

for record in vba_fauna.find():
    pprint(record)

{'_id': ObjectId('5f66480ac718f535282d620e'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.92259979999999,
 'long': 146.2884979,
 'record_id': '8590335',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2017, 8, 20, 0, 0),
 'start_mth': 8,
 'start_year': 2017,
 'survey_id': '1401349',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d620f'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8185997,
 'long': 145.9344025,
 'record_id': '6942095',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2015, 9, 8, 0, 0),
 'start_mth': 9,
 'start_year': 2015,
 'survey_id': '1102090',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6210'),
 'comm_name': 'Swift Parrot',
 'lat': -37.676399200000006,
 'long': 145.10839840000003,
 'record_id': '8596613',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2017, 9, 25, 0, 0)

 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2016, 3, 16, 0, 0),
 'start_mth': 3,
 'start_year': 2016,
 'survey_id': '1107769',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 5}
{'_id': ObjectId('5f66480ac718f535282d6252'),
 'comm_name': 'Hooded Plover',
 'lat': -37.8972015,
 'long': 147.8809967,
 'record_id': '9005526',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 2, 3, 0, 0),
 'start_mth': 2,
 'start_year': 2017,
 'survey_id': '1745931',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 7}
{'_id': ObjectId('5f66480ac718f535282d6253'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.782501200000006,
 'long': 146.1100006,
 'record_id': '6870922',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2015, 4, 17, 0, 0),
 'start_mth': 4,
 'start_year': 2015,
 'survey_id': '1085371',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282

 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6297'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8260002,
 'long': 145.797699,
 'record_id': '6948980',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2016, 2, 21, 0, 0),
 'start_mth': 2,
 'start_year': 2016,
 'survey_id': '1105996',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 2}
{'_id': ObjectId('5f66480ac718f535282d6298'),
 'comm_name': 'Mountain Pygmy-possum',
 'lat': -36.9547997,
 'long': 147.1578979,
 'record_id': '8490248',
 'sci_name': 'Burramys parvus',
 'start_date': datetime.datetime(2016, 11, 20, 0, 0),
 'start_mth': 11,
 'start_year': 2016,
 'survey_id': '1342577',
 'taxon_id': '11156',
 'taxon_type': 'Mammals',
 'totalcount': 45}
{'_id': ObjectId('5f66480ac718f535282d6299'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.729198499999995,
 'long': 146.16180419999998,
 'record_id': '6942306',
 'sci_name': 'Gymnobelideus leadbeateri',
 'st

{'_id': ObjectId('5f66480ac718f535282d62d8'),
 'comm_name': 'Mallee Emu-wren',
 'lat': -35.6450005,
 'long': 141.6976013,
 'record_id': '8986160',
 'sci_name': 'Stipiturus mallee',
 'start_date': datetime.datetime(2015, 11, 13, 0, 0),
 'start_mth': 11,
 'start_year': 2015,
 'survey_id': '1734055',
 'taxon_id': '10527',
 'taxon_type': 'Passerine birds',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d62d9'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.841499299999995,
 'long': 145.8031006,
 'record_id': '6870776',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2015, 1, 15, 0, 0),
 'start_mth': 1,
 'start_year': 2015,
 'survey_id': '1085276',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d62da'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.680198700000005,
 'long': 146.13209530000003,
 'record_id': '8584233',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(201

 'lat': -37.8540001,
 'long': 145.83569340000003,
 'record_id': '8206863',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2016, 1, 18, 0, 0),
 'start_mth': 1,
 'start_year': 2016,
 'survey_id': '1218229',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6322'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8446007,
 'long': 146.1195068,
 'record_id': '6871053',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2015, 2, 27, 0, 0),
 'start_mth': 2,
 'start_year': 2015,
 'survey_id': '1085434',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6323'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.3988991,
 'long': 145.82650759999999,
 'record_id': '8584346',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2017, 2, 2, 0, 0),
 'start_mth': 2,
 'start_year': 2017,
 'survey_id': '1398022',
 'taxon_id'

...

 'start_year': 2018,
 'survey_id': '1963031',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d68f4'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9978981,
 'long': 144.5917053,
 'record_id': '10093362',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2019, 4, 18, 0, 0),
 'start_mth': 4,
 'start_year': 2019,
 'survey_id': '1970693',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 40}
{'_id': ObjectId('5f66480ac718f535282d68f5'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9978981,
 'long': 144.5917053,
 'record_id': '10093385',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2019, 4, 18, 0, 0),
 'start_mth': 4,
 'start_year': 2019,
 'survey_id': '1970701',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 30}
{'_id': ObjectId('5f66480ac718f535282d68f6'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9978981,
 'long': 144.5917053,
 'record_id': '1009340

 'totalcount': 2}
{'_id': ObjectId('5f66480ac718f535282d693a'),
 'comm_name': 'Hooded Plover',
 'lat': -38.3232994,
 'long': 144.68299869999998,
 'record_id': '10038119',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 12, 28, 0, 0),
 'start_mth': 12,
 'start_year': 2017,
 'survey_id': '1948806',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 7}
{'_id': ObjectId('5f66480ac718f535282d693b'),
 'comm_name': 'Hooded Plover',
 'lat': -38.28599929999999,
 'long': 144.420105,
 'record_id': '9752298',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2015, 1, 1, 0, 0),
 'start_mth': 1,
 'start_year': 2015,
 'survey_id': '1908065',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d693c'),
 'comm_name': 'Hooded Plover',
 'lat': -38.3916016,
 'long': 142.2306976,
 'record_id': '10061490',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2018, 8, 24, 0, 0),
 'sta

 'lat': -37.8037987,
 'long': 146.06410219999998,
 'record_id': '10150678',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2020, 1, 28, 0, 0),
 'start_mth': 1,
 'start_year': 2020,
 'survey_id': '1988602',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6989'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.401199299999995,
 'long': 145.96000669999998,
 'record_id': '10179333',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2020, 1, 23, 0, 0),
 'start_mth': 1,
 'start_year': 2020,
 'survey_id': '2013903',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d698a'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.776199299999995,
 'long': 145.94200130000002,
 'record_id': '10134733',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2019, 11, 18, 0, 0),
 'start_mth': 11,
 'start_year': 2019,
 'surve

{'_id': ObjectId('5f66480ac718f535282d69c3'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8073006,
 'long': 146.11279299999998,
 'record_id': '10130964',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2019, 7, 25, 0, 0),
 'start_mth': 7,
 'start_year': 2019,
 'survey_id': '1982222',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d69c4'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8070984,
 'long': 146.11689760000002,
 'record_id': '10130975',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2019, 7, 25, 0, 0),
 'start_mth': 7,
 'start_year': 2019,
 'survey_id': '1982225',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d69c5'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.8059006,
 'long': 146.11450200000002,
 'record_id': '10130981',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(

 'start_mth': 12,
 'start_year': 2019,
 'survey_id': '2014689',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6a11'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.775600399999995,
 'long': 145.9732056,
 'record_id': '10183074',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2019, 12, 16, 0, 0),
 'start_mth': 12,
 'start_year': 2019,
 'survey_id': '2014694',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6a12'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.774601000000004,
 'long': 145.9734955,
 'record_id': '10183077',
 'sci_name': 'Gymnobelideus leadbeateri',
 'start_date': datetime.datetime(2019, 12, 17, 0, 0),
 'start_mth': 12,
 'start_year': 2019,
 'survey_id': '2014697',
 'taxon_id': '11141',
 'taxon_type': 'Mammals',
 'totalcount': 1}
{'_id': ObjectId('5f66480ac718f535282d6a13'),
 'comm_name': "Leadbeater's Possum",
 'lat': -37.467

In [38]:
from pprint import pprint

for record in scraped_fauna.find():
    pprint(record)

{'_id': ObjectId('5f66480bc718f535282d6a37'),
 'animal_name': 'Giant Burrowing Frog',
 'image_alternative': 'Giant Burrowing Frog on wet rocks side view. The frog '
                      'is dark brown with yellow lips and spots on its side.',
 'image_url': 'https://www.zoo.org.au/media/2056/23479_giant_burrowing_frog_-_credit_required_offsite1.jpg?anchor=center&mode=crop&quality=75&width=2000&height=570&rnd=132131643480000000',
 'introduction': 'Although we know that populations of the Giant Burrowing '
                 'Frog are in decline, this elusive digger is one of '
                 'Victoria’s most poorly understood species.',
 'threat_paragraph': "['Over the next five years, Zoos Victoria will carry out "
                     'important on-the-ground surveys to discover more about '
                     'the Giant Burrowing Frog and its remote habitat. In the '
                     'meantime, we are securing the wellbeing of the '
                     "individual frogs in our

In [39]:
type(vba_fauna.find())

pymongo.cursor.Cursor

In [40]:
type(scraped_fauna.find())

pymongo.cursor.Cursor

In [41]:
# agr = [
#   {
#     "$group" : {
#        "_id" : "$comm_name",
#        "totalSightings": { "$sum": "$totalcount" },
#     }
#   }
#  ]

In [42]:
# var = list(collection.aggregate(agr))
# var